# 第四章　Monte Carlo法：估算体积和计数

作者：[王何宇](http://person.zju.edu.cn/wangheyu)

[浙江大学数学科学学院](http://www.math.zju.edu.cn)

本文档配合讲义第四章。讲义中的代码和图形都可在此验证。

In [ ]:
from __future__ import print_function, division

# 这句话是非标准的python，用于ipthon或jupyter这样的系统中，表示绘图即刻自动展开。
%matplotlib inline

# 这里把全部Warning过滤掉了. 
# 参见https://docs.python.org/2/library/warnings.html
import warnings
warnings.filterwarnings('ignore')
from scipy.special import comb, perm
from scipy import stats
import numpy as np
import sys
import matplotlib.pyplot as plt
np.random.seed(250)

## 体积
下面是一个$m = 2$的单位正方形和均匀格点。

In [ ]:
def mesh_generator(k):
    zi = range(k)
    for z1 in zi:
        for z2 in zi:
            xi = (z1 + 1/2)/k
            yi = (z2 + 1/2)/k
            plt.scatter(xi, yi, color='b',marker='.', s=5)
    plt.xlim(0.0, 1.0)
    plt.ylim(0.0, 1.0)
    axs = plt.gca()
    axs.set_aspect('equal', 'box')
    return

产生一个$10 \times 10$的网格。

In [ ]:
mesh_generator(10)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel(r'x',fontsize=15) 
plt.xlabel(r'$x_1$',fontsize=15)
plt.ylabel(r'$x_2$',fontsize=15)
plt.savefig(r'lattice.pdf')

还记得我们之前学过的一个对正态分布的反函数做有理逼近的算法么？

In [ ]:
def inv_normal(y):
    c0 = 2.515517
    c1 = 0.802853
    c2 = 0.010328
    d1 = 1.432788
    d2 = 0.189269
    d3 = 0.001308
    t = np.sqrt(-np.log((min(y, 1 - y))**2))
    S1 = c0 + c1 * t + c2 * t**2
    S2 = 1 + d1 * t + d2 * t**2 + d3 * t**3
    S = t - S1 / S2
    w = y - 0.5
    if (w >= 0):
        F = S
    else:
        F = -S
    return F

我们用拟变换来生成一个正态分布采样看？

In [ ]:
def sample_Normal(U):
    return [inv_normal(u) for u in U]

In [ ]:
U = np.random.rand(1000000)
X = sample_Normal(U)

看一下采样效果。

In [ ]:
x = np.linspace(0,6,10000)
f = 1/np.sqrt(np.pi * 2) * np.exp(-x**2/2)
plt.hist(X, bins=200,normed=True, label="Stat. of the Sampling");
plt.plot(x, f, label=r"PDF")
plt.legend()
plt.xlabel(r"Statistical Test of Standard Samples by Approximation Inverse Transform Method")

似乎不错哦。然后我们再次回顾一下第一章的用于计算圆面积的程序。我们注意到它和MC算法其实是一致的。但是并没有给出统计检验部分。

In [ ]:
def area_circle(times):
    inside = 0
    dots = np.random.rand(2, times)
    for i in range(times):
        x = dots[0, i] - 0.5
        y = dots[1, i] - 0.5
        if x * x + y * y < 0.25:
            inside += 1
    return inside / times

我们现在来验证我们的统计估计。

如果我们的$n = 10000$，那么我们的误差会是多少？根据Chebshev不等式，我们可以计算
$$
n_C = \frac{1}{4 \delta \varepsilon^2},
$$
我们令$\delta = 0.01$，也就是说绝对误差小于$\varepsilon$的概率要大于百分之99。此时，
$$
\varepsilon = \sqrt{\frac{1}{4 n_C \delta}} = \sqrt{\frac{1}{4 \times 10000 \times 0.01}} = \frac{1}{20} = 0.05.
$$
我们实际计算一下误差的分布：

In [ ]:
T = 1000  
n = 10000
abs_err = np.zeros(T)
for i in range(T):
    abs_err[i] = (area_circle(n) - np.pi / 4)
plt.hist(abs_err, bins=50, normed=True, label="Stat. of the Sampling");

远好于我们的估计，说明我们的估计太保守了。现在来看利用大数定理的估计：
$$
n_N = \left[\frac{\Phi^{-1}(1 - \frac{\delta}{2})}{2 \varepsilon}\right]^2,
$$
同样令$\delta = 0.01$，则
$$
\varepsilon = \frac{\Phi^{-1}(0.995)}{\sqrt{2 n_N}}.
$$
这个我们用上面的有理逼近帮一下，当然正常的方式是查表。

In [ ]:
delta = 0.01
eps = inv_normal(1 - delta/2) / np.sqrt(2 * n)
print(eps)

这个估计要好的多。

In [ ]:
eps = np.sqrt(np.log(2/delta)/(2*n))
print(eps)

这个已经很准了。反过来，如果我们希望偏差在0.01（百分之一）以内的概率大于0.999，也即$\delta = 0.001$，那么用最后一个估计来计算，则至少需要样本数为：

In [ ]:
delta_fine = 0.001
eps = 0.01
2 * np.log(2 / delta_fine) / (4 * (eps)**2)

至少需要38005个样本点。

对于一些较难用数值方法计算的复杂的图形，比如：
$$
\int_0^1\sin(\frac{1}{x}) dx,
$$
用Monte Carlo法计算其曲线下面积（实际就是积分），毫无困难。我们甚至不需要对程序的算法部分做任何修改。先画一下我们要求面积的部分：

In [ ]:
x = np.logspace(-100, 0, 100000)
plt.plot(x, 0.5 * np.sin(1/x) + 0.5)

In [ ]:
x = np.linspace(0, 1, 100)
plt.plot(x, 0.5 * np.sin(1/x) + 0.5)

除了目标函数不一样，其他部分和求圆面积完全一样。

In [ ]:
def area_sin1_x(times):
    inside = 0
    dots = np.random.rand(2, times)
    for i in range(times):
        x = dots[0, i] - 0.5
        y = dots[1, i] - 0.5
        if y < 0.5 * np.sin(1/x) + 0.5:   # 只改了目标函数部分
            inside += 1
    return inside / times

先暴力一下给个参考解。

In [ ]:
lambda_n = area_sin1_x(1000000)
print(lambda_n)

连统计过程也一毛一样。可见MC方法对具体区域形状不敏感。这是它的另一个优势。

In [ ]:
T = 1000
abs_err = np.zeros(T)
for i in range(T):
    abs_err[i] = (area_sin1_x(n) - lambda_n)
plt.hist(abs_err, bins=50, normed=True, label="Stat. of the Sampling");

下面来看一下讲义中最后一个样本数估计如何实现。这里我们需要求解一个最大值，而且根据理论结果，$w$函数是凹的，也就是我们只要求其导数的零点就行。而为了求导数零点，我们可以构建一个Newton迭代，于是还要再求2阶导数。这个函数尽管是初等函数，但求导过程不会很愉快，我们用Python的符号计算扩展包：Sympy来帮助我们。下面就是调用Sympy，声明e和u为符号，然后定义w，并对w求一次导。

In [ ]:
import sympy as sym
e, u = sym.symbols('e,u', positive=True)
w = ((u + e) * sym.log(u/(u + e)) + (1 - u - e)* sym.log((1 - u)/(1 - u - e)))
sym.simplify(sym.diff(w, u))

再求其2阶导。

In [ ]:
sym.simplify(sym.diff(w,u,2))

注意我们已经调用了simplify函数，但大家注意到上面两个形式并不是最简。计算机对最简的理解和人类是不一样的。有节操的同学可以继续化简，不过这个只要等价对计算机也区别不大。但注意上面的形式都是符号计算，我们要转换成Python的函数以便于数值计算。复制粘帖下去，再把log改成np.log就行了。

In [ ]:
def hoeffding_w(e, u):
    return ((u + e) * np.log(u/(u + e)) + (1 - u - e)* np.log((1 - u)/(1 - u - e)))

In [ ]:
def hoeffding_dw(e, u):
    return ((-e*u + e*(u - 1) + u*(u - 1)*(np.log(u/(e + u)) - np.log((u - 1)/(e + u - 1))))/(u*(u - 1)))

In [ ]:
def hoeffding_d2w(e, u):
    return (e*(-u**2*(e + u)*(u - 1) + u**2*(e + u)*(e + u - 1) + u*(u - 1)**2*(e + u - 1) - (e + u)*(u - 1)**2*(e + u - 1))/(u**2*(e + u)*(u - 1)**2*(e + u - 1)))

人肉Newton迭代了解一下：

In [ ]:
eps=0.05
delta=0.01
u0=0.5
u1 = u0 - hoeffding_dw(eps,u0)/hoeffding_d2w(eps,u0)
err=np.abs(hoeffding_dw(eps,u1))
print(u1,err)

In [ ]:
u0=u1
u1 = u0 - hoeffding_dw(eps,u0)/hoeffding_d2w(eps,u0)
err=np.abs(hoeffding_dw(eps,u1))
print(u1,err)

In [ ]:
u0=u1
u1 = u0 - hoeffding_dw(eps,u0)/hoeffding_d2w(eps,u0)
err=np.abs(hoeffding_dw(eps,u1))
print(u1,err)

根据理论结果，此时，$w$取到最大值：

In [ ]:
print(hoeffding_w(eps,u1))

对应的有，关于$\varepsilon = 0.05$，$\delta = 0.01$的样本数量估计：

In [ ]:
n=-np.log(2/delta)/hoeffding_w(eps,u1)
print(n)

我们再对比一下之前的$\delta = 0.001$和$\varepsilon = 0.01$的估计：

In [ ]:
delta_fine = 0.001
eps = 0.01
u0=0.5
u1 = u0 - hoeffding_dw(eps,u0)/hoeffding_d2w(eps,u0)
err=np.abs(hoeffding_dw(eps,u1))
print(u1,err)

再次人肉Newton迭代（Newton迭代必须在几步内收敛）：

In [ ]:
u0=u1
u1 = u0 - hoeffding_dw(eps,u0)/hoeffding_d2w(eps,u0)
err=np.abs(hoeffding_dw(eps,u1))
print(u1,err)

此时$w$最大值为：

In [ ]:
print(hoeffding_w(eps,u1))

样本数量估计为：

In [ ]:
n=-np.log(2/0.001)/hoeffding_w(eps,u1)
print(n)

和之前的估计几乎一毛一样...（少了1）

接下去我们讨论区间估计。首先我们做一个实验，当$\lambda = 0.005$时，我们看一下实际上的模拟结果是如何分布的。为此我们还是调出之前编写的求圆面积的MC程序，反正MC方法对具体形状并不敏感。

In [ ]:
def area_circle_small(times):
    inside = 0
    dots = np.random.rand(2, times)
    for i in range(times):
        x = dots[0, i] - 0.5
        y = dots[1, i] - 0.5
        if x * x + y * y < 0.005 / np.pi:
            inside += 1
    return inside

我们可以观察到频数$S$大大降低。这是正常的。

In [ ]:
n = 1000
S = area_circle_small(1000)
V = S / n * (1 - S / n) / (n - 1)
print(S, V)

如果我们做多次模拟并加以统计，我们能够得到实际频率的分布。

In [ ]:
# 这个程序会很慢！
T = 50000  
n = 1000
abs_err = np.zeros(T)
for i in range(T):
    abs_err[i] = (area_circle_small(n) / n - 0.005)
plt.hist(abs_err, bins=40, normed=True, label="Stat. of the Sampling");
plt.legend()
plt.xlabel(r"Statistical Test of Frequency")
plt.savefig("bais_lambda_n.pdf")

上面的结果其实很好理解，因为$\lambda$本身就只有$0.005$，因此负方向最多只能再偏$0.005$，如果实际偏差会大于$\lambda$，那么在正方向和负方向就不会对称。事实上由于这种"边界"效应，除了$\lambda = 0.5$以外，置信区间都不会对称。

下面为了理解二项分布的累计分布函数，我们先从定义出发绘制，所以我们需要调用组合函数。

In [ ]:
# 调这个包为了计算组合数
from scipy.special import comb

计算10000取4。精确计算组合数是一个指数时间算法，在实际计算机模拟中不能接受。这里只是作为教学演示。然后我们会讨论如何避免直接计算。

In [ ]:
comb(10000,4)   

严格从定义出发构建二项分布的累积分布函数：

In [ ]:
def binomial_CDF(n, mu, i):
    F = 0
    for j in range(i + 1):
        F += comb(n, j) * mu**j * (1 - mu)**(n - j)
    return F

我们可以绘制一下它关于参数$\mu$的变化情况：

In [ ]:
x = np.linspace(0, 1, 100)
plt.plot(x, binomial_CDF(1000, x, 323))

上面是关于参数$\mu$的变化，下面是固定$n = 100$，$\mu = 0.5$的累计分布函数：

In [ ]:
x = [i for i in range(100)]
y = [binomial_CDF(100, 0.5, i) for i in range(100)]
plt.plot(x, y)

为了计算二项分布的累计分布函数的逆变换，我们将其转化为不完全Beta函数来计算：

In [ ]:
# 调用不完全Beta函数betainc和Beta函数
from scipy.special import betainc,beta

对比一下可以看到，不完全Beta函数可以替代二项分布累积分布函数。

In [ ]:
x = np.linspace(0, 0.1, 100)
plt.plot(x, 1-betainc(12, 1000-11+1, x))
plt.plot(x, binomial_CDF(1000, x, 11), 'o')
#plt.plot(x, (1-betainc(12,1000-12+1,x)-binomial_CDF(1000, x, 11)))

现在考虑如何求根，由于这个函数导数变化剧烈，但是单调，所以我们可以用二分法缩小含根区间。

In [ ]:
def func(S, n, delta, z):
    return betainc(S, n - S + 1, z) - delta / 2

def bisection_root(S, n, delta, z1, z2, err, tol):
    while (True):
        f1 = func(S, n, delta, z1)
        if np.abs(f1) < tol:
            return z1
        z = (z1 + z2) * 0.5
        fz = func(S, n, delta, z)
        if np.abs(fz) < tol:
            return z
        if fz * f1 < 0:
            z2 = z
        else:
            z1 = z
        z = (z1 + z2) * 0.5
        if (np.abs(z1 - z2) < err):
            return z

缩小到误差范围$1 \times 10^{-3}$。

In [ ]:
S = 11
n = 1000
delta = 0.01
z = bisection_root(S, n, delta, 0, 1, 1e-3, 1e-7)
print(z)

然后启动一个Newton迭代。这里我们认真写一个Newton迭代吧。首先注意到不完全Beta函数是一个变上限积分，因此它的导数就是积分核，也就是二项分布的质量分布函数。

In [ ]:
def dfunc(S, n, delta, z):
    return (z)**(S - 1)*(1 - z)**(n - S) / beta(S, n - S + 1)

In [ ]:
def Newton(S, n, delta, theta, tol, MAX_IT):
    theta0 = theta
    err = np.abs(func(S, n, delta, theta0))
    k = 0
    while err > tol:
        theta1 = theta0 - func(S, n, delta, theta0) / dfunc(S, n, delta, theta0)
        theta0 = theta1
        err = np.abs(func(S, n, delta, theta0))
        k = k + 1
        if k > MAX_IT:
            print ("Disconvergence!")
            break
    return theta0

In [ ]:
Newton(11, 1000, 0.01, 0.00439453125, 1e-15, 10)

下面我们对$S = 323$，$n = 1000$，$\delta = 0.01$利用上面的程序给出一个完整的区间估计。首先计算满足
$$
H_\theta(S, n - S + 1) = \frac{\delta}{2}
$$
的$\theta$：

In [ ]:
S = 323
n = 1000
delta = 0.01
z = bisection_root(S, n, delta, 0, 1, 1e-3, 1e-7)
print(z)

其实因为MC方法本身精度不会太高，这个估计就很好了，如果实在有必要，那么可以用Newton迭代继续加细：

In [ ]:
z = Newton(S, n, delta, z, 1e-15, 10)
print(z)

这是个神经病一样的估计。然后继续估计另一端，为此要重新定义一下func。

In [ ]:
def func(S, n, delta, z):
    return betainc(S + 1, n - S, z) - 1 + delta / 2
def dfunc(S, n, delta, z):
    return (z)**S*(1 - z)**(n - S - 1) / beta(S + 1, n - S)

In [ ]:
z = bisection_root(S, n, delta, 0, 1, 1e-3, 1e-7)
print(z)

In [ ]:
z = Newton(S, n, delta, z, 1e-15, 10)
print(z)

其实只要把二分法的精度提高也行，比如误差界$5 \times 10^{-5}$足以保证有4位有效数字：

In [ ]:
z = bisection_root(S, n, delta, 0, 1, 5e-5, 1e-7)
print(z)

抽取落在$\mathscr{R}_U \backslash \mathscr{R}_L$之间随机分布，也即服从$f(\vec{x}, \lambda_L, \lambda_U)$为概率密度函数的分布的方法有两个，一个是接受-拒绝方法（AR）：

In [ ]:
au1 = 0.2
bu1 = 0.8
al1 = 0.4
bl1 = 0.65
au2 = 0.1
bu2 = 0.9
al2 = 0.3
bl2 = 0.7
plt.plot([0,1,1,0,0],[0,0,1,1,0])
plt.plot([au1,bu1,bu1,au1,au1],[au2,au2,bu2,bu2,au2])
plt.plot([al1,bl1,bl1,al1,al1],[al2,al2,bl2,bl2,al2])
#plt.xlim(0.0, 1.0)
#plt.ylim(0.0, 1.0)
#plt.axis("equal")
axs = plt.gca()
axs.set_aspect('equal', 'box')
n = 1000
k = 0
T = 0
while (k < n):
    X0 = np.random.rand()
    Y0 = np.random.rand()
    if ((X0 > au1 and X0 < bu1) and (Y0 > au2 and Y0 < bu2)):
        if ((X0 < al1 or X0 > bl1) or (Y0 < al2 or Y0 > bl2)):
            plt.plot(X0, Y0, 'b.')
            k = k + 1
    T = T + 1
print(T)
Lu = (bu1 - au1) * (bu2 - au2)
Ll = (bl1 - al1) * (bl2 - al2)
print(1 / (Lu - Ll))

我们可以看到抽取效率比的期望为2.63，也就是每随机投点2.63次才能产生一个满足要求的投点。还有一种方法是直接抽取。我们将投点区域分割成(au1, au2)-(bu1,au2)-(bu1,al2)-(au1,al2)；(au1, bl2)-(bu1,bl2)-(bu1,bu2)-(au1,bu2)；(au1,al2)-(al1,al2)-(al1,bl2)-(au1,bl2)和(bl1,al2)-(bu1,al2)-(bu1,bl2)-(bl1,bl2)四块，它们的面积分别是：

In [ ]:
A = np.zeros(4)
A[0] = (bu1 - au1) * (bu2 - bl2)
A[1] = (al1 - au1) * (bl2 - al2)
A[2] = (bu1 - bl1) * (bl2 - al2)
A[3] = (bu1 - au1) * (al2 - au2)

其和为$\lambda_U - \lambda_L$。所以我们按这个质量分布进行投点，先决定投点落在那个分块：

In [ ]:
F = np.zeros(5)
for i in range(1, 5):
    F[i] = F[i - 1] + A[i - 1] 
F = F / (Lu - Ll)

def pick_area():
    Z = np.random.rand()
    for i in range(4):
        if Z < F[i + 1] and Z > F[i]:
            return i

def sample_A1():
    Z1 = np.random.rand()
    Z2 = np.random.rand()
    Z1 = au1 + Z1 * (bu1 - au1)
    Z2 = bl2 + Z2 * (bu2 - bl2)
    return (Z1, Z2)

def sample_A2():
    Z1 = np.random.rand()
    Z2 = np.random.rand()
    Z1 = au1 + Z1 * (al1 - au1)
    Z2 = al2 + Z2 * (bl2 - al2)
    return (Z1, Z2)

def sample_A3():
    Z1 = np.random.rand()
    Z2 = np.random.rand()
    Z1 = bl1 + Z1 * (bu1 - bl1)
    Z2 = al2 + Z2 * (bl2 - al2)
    return (Z1, Z2)

def sample_A4():
    Z1 = np.random.rand()
    Z2 = np.random.rand()
    Z1 = au1 + Z1 * (bu1 - au1)
    Z2 = au2 + Z2 * (al2 - au2)
    return (Z1, Z2)

plt.plot([0,1,1,0,0],[0,0,1,1,0])
plt.plot([au1,bu1,bu1,au1,au1],[au2,au2,bu2,bu2,au2])
plt.plot([al1,bl1,bl1,al1,al1],[al2,al2,bl2,bl2,al2])

axs = plt.gca()
axs.set_aspect('equal', 'box')
n = 1000
for i in range(n):
    I = pick_area()
    if (I == 0):
        X0, Y0 = sample_A1()
        plt.plot(X0, Y0, 'b.')
    if (I == 1):
        X0, Y0 = sample_A2()
        plt.plot(X0, Y0, 'b.')        
    if (I == 2):
        X0, Y0 = sample_A3()
        plt.plot(X0, Y0, 'b.')        
    if (I == 3):
        X0, Y0 = sample_A4()
        plt.plot(X0, Y0, 'b.')                

这个程序写的很烂，而且只针对2维，大家应该参照书上50页Version C自己考虑一下m维的情况该如何处理。这里顺便讨论一下在指定区域独立、均匀、随机投点的问题，考虑我们要再一个圆形区域投点。不妨设我们要投点的圆的半径$r_0$满足$0 < r_0 < 0.5$，圆心是$(0.5, 0.5)$，设$(X0, Y0)$是单位正方形内均匀分布的随机投点，则其到圆心的距离小于$r_0$则接受，否则拒绝。实现代码如下：

In [ ]:
n = 1000
X = np.zeros(n)
Y = np.zeros(n)
x0 = 0.5
y0 = 0.5
r = 0.5
i = 0
while (i < n):
    X0 = np.random.rand()
    Y0 = np.random.rand()
    if ((X0 - x0)**2 + (Y0 - y0)**2 <= r**2):
        X[i] = X0
        Y[i] = Y0
        i = i + 1
plt.plot(X, Y,'.')
plt.xlim(-0.1, 1.1)
plt.ylim(-0.1, 1.1)
#plt.axis("equal")
axs = plt.gca()
axs.set_aspect('equal', 'box')

第二个办法是用逆变换：假设我们要投点的圆，圆心是$(x_0, y_0)$，半径是$r_0$，设$(X, Y)$是圆内均匀分布的随机投点，则$(X, Y)$落在半径为$r$（$0 \leq r \leq r0$）的**概率密度**是
$$
f(r) = c*2*\pi*r,
$$
其中$c$是归一化系数。于是对应的累积分布函数为：
$$
F(r) = c*\pi*r^2. 
$$
代入$F(r_0) = 1$，得：
$$
c * \pi * r_0^2 = 1 \rightarrow c = \frac{1}{\pi * r_0^2},
$$
代入得：
$$
F(r) = \frac{1}{\pi * r_0^2} * \pi *r^2 = \frac{r^2}{r_0^2}.
$$
于是，令$s = F(r)$，则其逆变换为：
$$
F^{-1}(s) = \sqrt{r_0^2 * s}.
$$
也即第一步我们先做一个$s \sim U(0, 1)$的均匀采样，然后用$R = F^{-1}(s)$得到我们在投点范围圆内的半径。接下去考虑幅角，这个自然是$T \sim U(0, 2\pi)$. 直接产生这个均匀采样。所求圆内的均匀采样就是：
$$
\left\{
\begin{array}{rcl}
X &=& R * \cos(T) + x_0,\\
Y &=& R * \sin(T) + y_0.\\
\end{array}
\right.
$$
实现代码如下：

In [ ]:
n = 1000
X = np.zeros(n)
Y = np.zeros(n)
x0 = 0.5
y0 = 0.5
r = 0.5
i = 0
for i in range(n):
    R = np.sqrt(np.random.rand() * r**2)
    T = 2 * np.pi * np.random.rand()
    X[i] = R * np.cos(T) + x0 
    Y[i] = R * np.sin(T) + y0
plt.plot(X, Y,'.')
plt.xlim(-0.1, 1.1)
plt.ylim(-0.1, 1.1)
#plt.axis("equal")
axs = plt.gca()
axs.set_aspect('equal', 'box')    